<center><img src='https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles/blob/main/images/logo.png?raw=true'/></center>

<h1 align = 'center'> Trabalho de Circuitos Elétricos - II<h1/>
<h2 align = 'center'> Simulação de Sistemas Elétricos por Quadripolos <h2\>

Este projeto se refere ao estudo sobre dimensionamento de filtros ativos, no qual o mesmo está salvo e disponível para consulta no seguinte [repositório](https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles).

O projeto foi realizado pelos seguinte alunos de Engenharia Elétrica da UFCG:

- Breno Henrique Martins Silva;
- Caíque Moraes de Lima Mangabeira;
- Emerson Matheus Lima da Silva
- Marcos Eduardo Araújo  
- Marcus Vinicius Almeida Filho

---

Utilizando dos conteúdos repassados nas aulas e também nos livros textos que se retratam sobre teoria de quadripolos, iremos utilizar a modelagem de dos componentes da seguinte forma:

### Linha de Transmissão de modelo $\pi$

<center><img src='https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles/blob/main/images/linha.png?raw=true'/></center>

Linhas de transmissão são estruturas físicas utilizadas para transportar energia elétrica de uma fonte (geradores) até os consumidores finais (residências, indústrias, etc.) através de longas distâncias. As principais carácteristicas das Linhas de Transmissão, que estão distribuidas por toda a linha, são a sua Impedância Característica $Z = R + jX_L$ que influência diretamente em como a energia é transmitida e distribuída ao longo da linha, e sua admitância $Y = jX_C$ a admitância é um fator importante que modela o efeito da capacitância e das perdas dielétricas em linhas de transmissão. Seu impacto é notado especialmente em linhas longas, onde contribui para a corrente reativa e pode afetar a regulação de tensão e o fluxo de potência em sistemas de transmissão e distribuição de energia elétrica.

As Linhas de Transmissão de modelo $\pi$ possuem um comprimento de 80 a 250 quilômetros, e para que possamos trabalhar com este modelo devemos concentrar seus parâmetro em uma equação matricial, no qual a tensão e corrente no final e início da linhas de transmissão possam ser determinadas, assim utilizamos a seguinte matriz para representar o modelo do tipo $\pi$ para linhas de transmissão:

$$
\begin{bmatrix}
V_1\\
I_1
\end{bmatrix} = \begin{bmatrix}
1 + \frac{ZY}{2} & Z\\
Y+\frac{ZY^{2}}{4} & 1 + \frac{ZY}{2}
\end{bmatrix} \cdot \begin{bmatrix}
V_2\\
I_2
\end{bmatrix}
$$
---

### Transformador

<center><img src='https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles/blob/main/images/transformador.png?raw=true'/></center>

Iremos utilizar um transformador de um sistema elétrico de potência com o modelo como mostrado acima, e assim como nas Linhas de Transmissão ele também possui uma matriz que representa a relação entre tensão e corrente nos seus terminais, utilizando o conceito de um transformador ideal temos que a sua matriz de transformação é da seguinte forma:

$$
\begin{bmatrix}
n & 0\\
0 & \frac{1}{n}
\end{bmatrix}
$$

Utilizando a matriz de transformação de um Transformador Ideal em conjunto com o modelo de matriz de uma linha de transmissão obteremos a seguinte matriz de transformação:

$$
\begin{bmatrix}
V_1\\
I_1
\end{bmatrix} = \begin{bmatrix}
n(1+YZ_1) &\frac{1}{n}(Z_1+Z_2+YZ_1Z_2) \\
nY & \frac{1}{n}(1+YZ_2)
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\
I_2
\end{bmatrix}
$$

Em que:

$$ Z_1 = R_1 + jX_{L_1}$$
$$ Z_2 = R_2 + jX_{L_2}$$
$$Y = \frac{R_m + jX_m}{R_m \cdot jX_m}$$

---

### Carga em Série

A tensão e corrente nos terminais de uma carga colocada em série com o circuito é dada por:

$$\begin{bmatrix}
V_1\\
I_1
\end{bmatrix} = \begin{bmatrix}
1 &Z \\
0 & 1
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\
I_2
\end{bmatrix} $$

---

### Carga em Paralelo

A tensão e corrente nos terminais de uma carga colocada em paralelo com o circuito é dada por:

$$ \begin{bmatrix}
V_1\\
I_1
\end{bmatrix} = \begin{bmatrix}
1 &0 \\
Y & 1
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\
I_2
\end{bmatrix}$$

---

In [856]:
# Importando as Bibliotecas
import numpy as np
import cmath

In [857]:
# Declaração de todas as funções:

def MatrixSeriesImpedance(Z):                                   # Retorna a matriz referente a uma impedância em série.
    return np.array([[1, Z], [0, 1]])

def MarixShuntAdimitance(Y_C):                                  # Retorna a matriz referente a carga em derivação.
    return np.array([[1, 0], [Y_C, 1]])

def MatrixTCircuit(Z1, Z2, N, Ym, Tap=1):                              # Retorna a matriz referente ao Transfomador.
    # Calculando a matriz a partir da multiplicação do circuito T com transformador ideal.
    A = (N * Tap) * (1 + (Ym * Z1))
    B = (1/(N * Tap)) * (Z1 + Z2 + (Ym * Z1 * Z2))
    C = (N * Tap) * (Ym)
    D = (1/(N * Tap)) * (1 + (Ym * Z2))

    return np.array([[A, B], [C, D]])

def MatrixRCircuit(Z, Y_L):                                     # Retorna a matriz referente a linha de transmissão.
    A = (1 + ((Z * Y_L) / 2))
    B = Z
    C = (Y_L + ((Z * Y_L**2) / 4))
    D = (1 + ((Z * Y_L) / 2))

    return np.array([[A, B], [C, D]])

def MatrizParallelCircuit(Matrix_1, Matrix_2):                  # Retorna a Matriz de associação em pararelo de dois circuitos.
    A1, B1, C1, D1 = Matrix_1[0,0], Matrix_1[0,1], Matrix_1[1,0], Matrix_1[1,1]
    A2, B2, C2, D2 = Matrix_2[0,0], Matrix_2[0,1], Matrix_2[1,0], Matrix_2[1,1]
    # Calculando a matriz para circuitos paralelos.
    A = (((A1 * B2) + (A2 * B1)) / (B1 + B2))
    B = ((B1 * B2) / (B1 + B2))
    C = (C1 + C2 + (((A1 - A2) * (D2 - D1)) / (B1 + B2)))
    D = (((B2 * D1) + (B1 * D2)) / (B1 + B2))

    return np.array([[A, B], [C, D]])

def MatrizCascadeCircuits(matrix):
    result = matrix[0]
    for m in range(len(matrix) - 1):
        result = np.dot(result, matrix[m+1])
    return result

# Tranformando em forma polar para pegar o fasorial.
def fasor(v):
    return f'{np.abs(v):.4f}∠{np.angle(v, deg=True):.4f}°'


### Questão 1

<center><img src='https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles/blob/main/images/SEP.png?raw=true'/></center>

Considerando o sistema elétrico da figura acima, no qual está operando com $60 \text{hz}$, modele o sistema criando as matrizes de transferência $[T]$ a partir dos dados de entrada do modelo que está logo abaixo:

**Impedância de Thévenin**
- $ R_f = 4 \Omega$;
- $ Xf = 0.38 \Omega$.

**Parâmetros das Linhas de Transmissão**
- $\rho_R = 0.182 \Omega/km$;
- $\rho_L = 2.28 mH/km$;
- $\rho_C = 0.0140 \mu F/km$.

Considere que as $LT_1 , LT_2 , LT_3$ possuem 100 km, a $LT_4$ e $LT_5$ possui 80km e a $LT_6$ possui 120 km.

**Parâmetros comum dos Transformadores**
- $R_1 = 7.6m \Omega$;
- $X_1 = 3.8m \Omega$;
- $R_2 = 33.9m \Omega$;
- $X_2 = 0.85m \Omega$.

**Parâmetros individuais dos Transformadores**
- Para o Transformador 1:
    - $R_{m_1} = 4320 \Omega$ e $X_{m_1} = 5050 \Omega$;
- Para o Transformador 2:
    - $R_{m_2} = 432000 \Omega$ e $X_{m_2} = 505000 \Omega$;
- Para o Transformador 3:
    - $R_{m_3} = 402000 \Omega$ e $X_{m_3} = 607000 \Omega$.

**Cargas em Derivação**

- $R_1 = 8400 \Omega$ e $L_1 = 46H$;
- $R_2 = 1175.55 \Omega$ e $L_2 = 6.43H$;
- $R_3 = 526 \Omega$ e $L_3 = 2.9H$.

In [858]:
# Declaração das variáveis e chamando as funções:

w = 2 * np.pi * 60                  # Declaração do nosso omega.

# Declaração da Impedância de Thevenin.
Rf, Xf = 4, 0.38j
Zf = Rf + Xf
mImpedances = MatrixSeriesImpedance(Zf)

# Parametros comum aos transformadores:
R1 = 7.6e-3
X1 = 3.8e-3
R2 = 33.9e-3
X2 = 0.85e-3

ZT1 = R1 + (X1 * 1j)
ZT2 = R2 + (X2 * 1j)

# Declaração do Transformador 1:
Rm1 = 4320
Xm1 = 5050j
N1 = 69/500
Y_T1 = ((Rm1 + Xm1) / (Rm1 * Xm1))
mT1 = MatrixTCircuit(ZT1, ZT2, N1, Y_T1, 1)

# Parametros comum para as linhas de transmissão :
RLT = 0.182
LLT = w * 2.28e-3 * 1j
CLT = w * 0.0140e-6 * 1j
ZLT = (RLT + (LLT))
Y_L = CLT/2

# Parametros da linha de transmissão LT1, LT2.
d1 = 100 #km - Distanciamento das linhas de transmissão.
mLT1 = MatrixRCircuit(ZLT*d1, Y_L*d1)
mLT2 = MatrixRCircuit(ZLT*d1, Y_L*d1)

# Parametros da carga Z1.
RC1 = 8400
LC1 = 46
Y1 = (1 / (RC1 + (LC1 * w * 1j)))
mZ1 = MarixShuntAdimitance(Y1)

# Parametros da linha de transmissão LT3.
mLT3 = MatrixRCircuit(ZLT*d1, Y_L*d1)

# Declaração do Transformador 2:
Rm2 = 432000
Xm2 = 505000j
N2 = 500/230
Y_T2 = ((Rm2 + Xm2) / (Rm2 * Xm2))
mT2 = MatrixTCircuit(ZT1, ZT2, N2, Y_T2, 1)

# Parametros da carga Z2.
RC2 = 1175.55
LC2 = 6.43
Y2 = (1 / (RC2 + (LC2 * w * 1j)))
mZ2 = MarixShuntAdimitance(Y2)

# Parametros da linha de transmissão LT4
d2 = 80 #km - Distanciamento das linhas de transmissão.
mLT4 = MatrixRCircuit(ZLT*d2, Y_L*d2)

# Declaração do Transformador 3:
Rm3 = 402000
Xm3 = 607000j
N3 = 230/69
Y_T3 = ((Rm3 + Xm3) / (Rm3 * Xm3))
mT3 = MatrixTCircuit(ZT1, ZT2, N3, Y_T3, 1)

# Parametros da carga Z3.
RC3 = 529
LC3 = 2.9
Y3 = (1 / (RC3 + (LC3 * w * 1j)))
mZ3 = MarixShuntAdimitance(Y3)

# Associação em pararelo das linhas de Transmissão LT1 e LT2:
mLT1T2 = MatrizParallelCircuit(mLT1, mLT1)

# Calculando a matriz de tranferência total do circuito:
mTotal = np.array([mImpedances, mT1, mLT1T2, mZ1, mLT3, mT2, mZ2, mLT4, mT3])

totalM = MatrizCascadeCircuits(mTotal)

print(totalM)

[[9.64085178e-01+0.16259615j 4.59932720e+00+9.09454221j]
 [3.01796834e-03+0.03667228j 7.19549147e-01+0.08206635j]]


## Montagem no Ambiente virtual - *LTSpice*

Fazendo uso do modelo apresentado anteriormente para os elementos do sistema de potência, utilizando o simulador *LTSpice* para simular todo o sistema e tornar-se possível a comparação com resultados encontrados na modelagem utilizando a linguagem de programação *Python*.

<center><img src='https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles/blob/main/images/circuit-simulation.png?raw=true'/></center>

Para realizar a montagem do modelo ideal de um transformador no simulador foi utilizado a função *"SPICE Directive"* na qual simula funções matemáticas em favor de alguns elementos existêntes no circuito, sendo assim utilizado para fazer com que os indutores dos transformadores estejam devidamente acoplados. Logo em seguida está o modelo de um transformador ideal montado no simulador *LTSpice* seguindo o modelo de transformado especificado anteriormente.

<center><img src='https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles/blob/main/images/simulated-transformer.png?raw=true'/></center>

Com todos as implementações realizadas no simulador, a modelagem do sistema por quadripolos aprensentou resultados similar a análise por elementos de circuito simulados no *LTSpice*.

### Item 1

Obtenha a tensão fasorial de saída $V_{ac}$ e a corrente fasorial na carga $Z_3$ considerando que a tensão no gerador seja de $69 kV_{RMS}$.

$R$: Nos quadripolos, a relação entre a tensão e corrente de entrada (lado da fonte) e de saída (lado da carga) é dada por:

$$\begin{bmatrix}
V_{fonte}\\
I_{fonte}
\end{bmatrix} = \begin{bmatrix}
A & B \\
C & D
\end{bmatrix}\cdot \begin{bmatrix}
V_{carga}\\
I_{carga}
\end{bmatrix} $$


Sabemos que a corrente e a tensão em ($Z_3$) estão relacionadas por sua **impedância**:

$$ I_3 = \frac{V_3}{Z_3}$$

Substituindo $I_3 = \frac{V_3}{Z_3}$ na equação do quadripolo:

$$ V_1 = A \cdot V_3 + B \cdot \frac{V_3}{Z_3} $$

Isolando $(V_{3})$:   $$V_3 = \frac{V_1 \cdot Z_3}{A \cdot Z_3 + B} $$

In [859]:
# A,B,C,D são os elementos da matriz de transferência total.
mA = totalM[0, 0]
mB = totalM[0, 1]
mC = totalM[1, 0]
mD = totalM[1, 1]

Vac = 69e3
Z3 = 1/Y3
VZ3 = (Vac * Z3) / ((mA * Z3) + mB)
IZ3 = VZ3/Z3

print(f'A tensão Vac é igual a {fasor(Vac)} V')
print(f'\nTensão e Corrrente Fasorial na Carga Z3: \n• VZ3 = {fasor(VZ3)} V\n• IZ3 = {fasor(IZ3)} A')

A tensão Vac é igual a 69000.0000∠0.0000° V

Tensão e Corrrente Fasorial na Carga Z3: 
• VZ3 = 69983.2641∠-9.4835° V
• IZ3 = 57.6216∠-73.6627° A


### Item 2

Obtenha a tensão e a corrente fasoriais nas impedâncias $Z_1$ e $Z_2$.

$R$: Vamos usar a matriz de transferência total até as cargas que inclui as linhas de transmissão, transformadores e as impedâncias:


$$\begin{bmatrix}
V_{fonte}\\
I_{fonte}
\end{bmatrix} = \begin{bmatrix}
A & B \\
C & D
\end{bmatrix}\cdot \begin{bmatrix}
VZ\\
IZ
\end{bmatrix} $$

In [860]:
# Calculando os valores de tensão e corrente de Z1:
mtotal = np.array([mImpedances, mT1, mLT1T2])
mTZ1 = MatrizCascadeCircuits(mtotal)

Z1 = 1 / Y1

VZ1 = (Z1 * Vac) / (mTZ1[0, 0] * Z1 + mTZ1[0, 1])

IZ1 = VZ1 / Z1

print(f'Tensão e Corrrente Fasoriais na Carga Z1: \n• V1 = {fasor(VZ1)} V\n• I1 = {fasor(IZ1)} A')

Tensão e Corrrente Fasoriais na Carga Z1: 
• V1 = 503864.0580∠-6.0320° V
• I1 = 26.1491∠-70.1872° A


In [861]:
# Calculando os valores de tensão e corrente de Z2:
mtotal = np.array([mImpedances, mT1, mLT1T2, mZ1, mLT3, mT2])
mTZ2 = MatrizCascadeCircuits(mtotal)

Z2 = 1 / Y2

VZ2 = (Vac * Z2) / (mTZ2[0, 0] * Z2 + mTZ2[0, 1])

IZ2 = VZ2 / Z2

print(f'Tensão e Corrrente Fasoriais na Carga Z2: \n• V2 = {fasor(VZ2)} V\n• I2 = {fasor(IZ2)} A')

Tensão e Corrrente Fasoriais na Carga Z2: 
• V2 = 232428.2312∠-8.8363° V
• I2 = 86.2744∠-72.9651° A


### Item 3

Obtenha a potência $Ativa$ e $Reativa$ fornecida pelo Gerador.

$R$: A potência ativa refere-se à energia útil, enquanto a potência reativa está associada ao armazenamento temporário de energia nos componentes reativos (indutâncias e capacitâncias).

A relação entre potência ativa, reativa e aparente é descrita pelo triângulo de potência:
$$ S^2 = P^2 + Q^2 $$

E utlizando deste conhecimento e também dos valores ja calculados anteriormente podemos encontrar a nossa *Potência Complexa* e extrair os valores da *Potencia Ativa e Reativa*, utilizando das seguintes equações:

$$ S_{complexo} = V_{RMS} \cdot \overline{I_{RMS}}  $$

In [862]:
# Calculando a corrente do gerador:
Iac = (mC * VZ3) + (mD * IZ3)

Vac = (mA * VZ3) + (mB * IZ3)
absIac, angIac =  cmath.polar(Iac)

# Calculando a potencia Ativa e Reativa:
Sg = Vac * (complex(absIac, - angIac))
Pg = Sg.real / pow(10,3)
Qg = Sg.imag / pow(10,3) 

print(f"A Potência Ativa no gerador é: {Pg:.4f} kW \nPotência Reativa é: {Qg:.4f} kVAr")

A Potência Ativa no gerador é: 175393.3840 kW 
Potência Reativa é: -90.6163 kVAr


### Item 4

Obtenha a potência $Ativa$ e $Reativa$ consumida pelas cargas.

$R$: A potência ativa refere-se à energia útil, enquanto a potência reativa está associada ao armazenamento temporário de energia nos componentes reativos (indutâncias e capacitâncias).

A relação entre potência ativa, reativa e aparente é descrita pelo triângulo de potência:
$$ S² = P² + Q² $$

Assim da mesma forma como no item aterior também podemos fazer uso da equação do $S_{complexo} = V_{RMS} \cdot \overline{I_{RMS}}$ para que possamos calcular as *Potências Ativas e Reativas* das cargas $Z_1 , Z_2$ e $Z_3$

In [863]:
# Calculando a potência complexa das Cargas Z1, Z2 e Z3

sZ1 = VZ1 * np.conj(IZ1) 
sZ2 = VZ2 * np.conj(IZ2)
sZ3 = VZ3 * np.conj(IZ3)

# Extraindo os valores da Potência Ativa das Cargas

pZ1 = sZ1.real / pow(10,3)
pZ2 = sZ2.real / pow(10,3)
pZ3 = sZ3.real / pow(10,3)

# Extraindo os valores da Potência Reativa das Cargas

qZ1 = sZ1.imag / pow(10,3)
qZ2 = sZ2.imag / pow(10,3)
qZ3 = sZ3.imag / pow(10,3)
 
print(f'A Potência Ativa na Carga Z1 é de {pZ1:.4f} kW e a Potência Reativa é igual a {qZ1:.4f} kVAr')
print(f'A Potência Ativa na Carga Z2 é de {pZ2:.4f} kW e a Potência Reativa é igual a {qZ2:.4f} kVAr')
print(f'A Potência Ativa na Carga Z3 é de {pZ3:.4f} kW e a Potência Reativa é igual a {qZ3:.4f} kVAr')

A Potência Ativa na Carga Z1 é de 5743.7015 kW e a Potência Reativa é igual a 11857.7292 kVAr
A Potência Ativa na Carga Z2 é de 8749.9360 kW e a Potência Reativa é igual a 18042.8800 kVAr
A Potência Ativa na Carga Z3 é de 1756.4091 kW e a Potência Reativa é igual a 3629.9372 kVAr


### Item 5

Calcule a perda de *Potência Ativa* e *Reativa* no sistema.

Para calcular a perda de potência do sistema realizamos a soma de todas as potências das cargas encontradas anteriormente e subtraímos da potência complexa do gerador assim como representa as equações abaixo:

$$

S_{perda} = S_{gerador} - (S_{Z_{1}} + S_{Z_2} + S_{Z_3})

$$

In [864]:
# potencia total da cargas

S_total_cargas = sZ1 + sZ2 + sZ3

#perda de potencia Ativa e Reativa no sistema.

S_perda = (Sg - S_total_cargas)

perdaP = S_perda.real / pow(10,6)
perdaQ = S_perda.imag / pow(10,6)

print(f'A perca de Potência Ativa no sistema é de {perdaP:.4f} MW \nE a perca de Potência Reativa no sistema é de {perdaQ:.4f} MVAr')

A perca de Potência Ativa no sistema é de 159.1433 MW 
E a perca de Potência Reativa no sistema é de -33.6212 MVAr


### Item 6 

Qual é o ajuste do TAP dos transformadores $T_1$, $T_2$ e $T_3$ para que as tensões nas cargas sejam, respectivamente, 500 kV, 230 kV e 69 kV.

O *TAP*, Tomada de Ajuste de Potência, que é uma característica de transformadores que permite ajustar a relação de transformação, no qual altera a tensão do lado secundário sem precisar desligar o transformador. Dessa forma utilizamos as equações ja demonstradas anteriormente para refazer os cálculos das tensões das cargas que estão em nosso sitema ja com o valor reajustado utilizando o TAP, a fim de que retorne o valor mais aproximado o possível do que é requerido.

In [865]:
MT1adj = MatrixTCircuit(ZT1, ZT2, N1, Y_T1, 1.00781657)
MT2adj = MatrixTCircuit(ZT1, ZT2, N2, Y_T2, 1.00313564)
MT3adj = MatrixTCircuit(ZT1, ZT2, N3, Y_T3, 1.0037527)
V1 = 69E3

#Calculando VZ1 depois do ajuste
M_Z1adj = np.array([mImpedances, MT1adj, mLT1T2])
MF_Z1adj = MatrizCascadeCircuits(M_Z1adj)

VZ1adj =  (Z1 * Vac) / (MF_Z1adj[0, 0]*Z1 + MF_Z1adj[0, 1])

absV1adj, angV1adj =  cmath.polar(VZ1adj)
angV1adj = np.rad2deg(angV1adj)

print(f"VZ1 = {absV1adj:.4f} ∠ {angV1adj:.4f}° V")

#Calculando VZ2 depois do ajuste
M_Z2adj = np.array([mImpedances, MT1adj, mLT1T2, mZ1, mLT3, MT2adj])
MF_Z2adj = MatrizCascadeCircuits(M_Z2adj)

VZ2 = (V1*Z2) / (MF_Z2adj[0, 0]*Z2 + MF_Z2adj[0, 1])

absV2, angV2 =  cmath.polar(VZ2)
angV2 = np.rad2deg(angV2)

print(f"VZ2 = {absV2:.4f} ∠ {angV2:.4f}° V")

#Calculando VZ3 depois do ajuste
M_Z1adj = np.array([mImpedances, MT1adj, mLT1T2, mZ1, mLT3, MT2adj, mZ2, mLT4, MT3adj])
MF_Z3adj = MatrizCascadeCircuits(M_Z1adj)

A = MF_Z3adj[0, 0]
B = MF_Z3adj[0, 1]
C = MF_Z3adj[1, 0]
D = MF_Z3adj[1, 1]


Z3 = 1/Y3
VZ3 = (V1 * Z3) / (A*Z3 + B)
I3 = VZ3/Z3

absV3, angV3 =  cmath.polar(VZ3)
angV3 = np.rad2deg(angV3)

print(f"VZ3 = {absV3:.4f} ∠ {angV3:.4f}° V")

VZ1 = 500000.0006 ∠ -5.9411° V
VZ2 = 230000.0012 ∠ -8.7150° V
VZ3 = 69000.0014 ∠ -9.3546° V
